In [14]:
from postprocess_base import read_logs
import altair as alt
from glob import glob

runs = read_logs(glob('noether-64/petsc-bps-2020-*.txt'))
runs['FE_nodes_per_compute_node'] = runs['num_unknowns'] / (runs['num_procs'] / runs['num_procs_node']) / runs['dof_per_node']
runs.head()

,file,backend,backend_memtype,hostname,test,num_procs,num_procs_node,degree,quadrature_pts,code,bp,case,num_unknowns,num_elem,dof_per_node,ksp_its,time_per_it,cg_iteration_dps,FE_nodes_per_compute_node
0,noether-64/petsc-bps-2020-06-09.1.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 1,64,64,1,3,libCEED,1,scalar,55473,51712,1,20,0.000152,366007000.0,55473.0
1,noether-64/petsc-bps-2020-06-09.1.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 2,64,64,1,3,libCEED,2,vector,166419,51712,3,20,0.000257,648499000.0,55473.0
2,noether-64/petsc-bps-2020-06-09.1.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 3,64,64,1,3,libCEED,3,scalar,47151,51712,1,1,0.000248,189977000.0,47151.0
3,noether-64/petsc-bps-2020-06-09.1.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 4,64,64,1,3,libCEED,4,vector,141453,51712,3,1,0.000495,285513000.0,47151.0
4,noether-64/petsc-bps-2020-06-09.1.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 1,64,64,1,3,libCEED,1,scalar,109395,100800,1,20,0.000285,383225000.0,109395.0


In [12]:
highlight = alt.selection_single(
    on='mouseover',
    fields=['degree', 'time_per_it', 'backend'],
    nearest=True,
    empty='none',
)

bps_select = alt.selection_single(
    fields=['bp'],
)

base = alt.Chart(runs).encode(
    alt.Y('mdofs:Q', title='MDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns', 'file'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e6',
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5)),
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(2), alt.value(1))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', scale=alt.Scale(type='log'), title='Time per Iteration'),
    ) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True), title='FE Nodes per Compute Node'),
    )
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='CEED BP4 - Noether (2x EPYC 7452), gcc-10')

alt.HConcatChart(...)